In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re
from numpy import random
import pprint
import queue as que
from queue import LifoQueue
import operator
import collections

#reference kd-tree split value to github: https://github.com/HolyChen/KdTree/blob/master/kdtree.py

#Prepocess Data

* Mounted Google Drive
* Create path to folder
* Load data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/MyDrive/AI-Future/LocalSensitiveHashing/"

#definition of class POINT
* Definition

In [3]:
class Point:
      def __init__(self,*args):
          if len(args) == 0:
             self.x = 0.0
             self.y = 0.0
             self.z = 0.0
          elif len(args) == 1 and isinstance(args[0],Point):
             self.x = args[0].x
             self.y = args[0].y
             self.z = args[0].z
          else:
             self.x = args[0]
             self.y = args[1]
             self.z = args[2]

      def set_x(self,x):
          self.x = x
      
      def set_y(self,y):
          self.y = y
      
      def set_z(self,z):
          self.z = z

In [ ]:
#gen 3-d data
n = int(input())
f = open('a.txt',"w")
f.write("{} \n".format(n))

for i in range(0,n):
    l_gen = []
    for j in range(0,3):
        g = round(random.uniform(0,100),3)
        l_gen.append(g)
    if i != n - 1:
        f.write("{} {} {}".format(l_gen[0],l_gen[1],l_gen[2]))
        f.write('\n')
    else:
        f.write("{} {} {}".format(l_gen[0],l_gen[1],l_gen[2]))

f.close()

10


In [ ]:
#preprocess-data => convert to 3d vector for local hashing
# path_data = path + 'a.txt'
points = []
with open('a.txt','r',encoding='UTF-8') as f:
     n = int(f.readline())
     for i in range(n):
         line = f.readline()
         lst = re.split("\s+",line)
         if len(lst) >= 4:
            lst = lst[:-1]
         point = Point()
         point.set_x(round(float(lst[0]),3))
         point.set_y(round(float(lst[1]),3))
         point.set_z(round(float(lst[2]),3))

         points.append(point)      

#Definition of class Node

In [ ]:
class Node_normal:
      def __init__(self,point):
          self.data = point
          self.left = None
          self.right = None

#Helper Function

In [ ]:
def euclide_distance(a,b):
    point1 = a
    point2 = b

    dist = 0.0

    dist = pow(point1.x - point2.x,2) + pow(point1.y - point2.y,2) + pow(point1.z - point2.z,2)

    dist = np.sqrt(dist)
    
    return dist

def preprocess(points,level,begin,end):
    if begin >= end:
       return None
    check = level % 3
    if check == 0:
       points[begin : end] = sorted(points[begin : end], key=operator.attrgetter('x'))
    elif check == 1:
       points[begin : end] = sorted(points[begin : end], key=operator.attrgetter('y'))
    else:
       points[begin : end] = sorted(points[begin : end], key=operator.attrgetter('z'))
    mid = int((begin + end)/2)
    preprocess(points,level + 1,begin,mid)
    preprocess(points,level + 1,mid + 1,end)

#RecFunction


*   checkPoint(a,b)
*   insertRec(root,data,level)
*   searchRec(root,point,level)
*   deleteRec(root,point,level)
*   print2DUtil(root,space = 0) -> None
*   clearRec(root) -> None



In [ ]:
#==============================================================================
def checkPoint(a,b):
    if a.x != b.x:
       return False
    if a.y != b.y:
       return False
    if a.z != b.z:
       return False
    return True
#==============================================================================

#==============================================================================
def insertRec(root,data,level):
    if root is None:
       return Node_normal(data)
    check = level % 3
    root_cor = 0.0
    data_cor = 0.0

    #gen_data of root_cor and data_cor    
    if check == 0:
       root_cor = root.data.x
       data_cor = data.x
    elif check == 1:
       root_cor = root.data.y
       data_cor = data.y
    else:
       root_cor = root.data.z
       data_cor = data.z

    if root_cor > data_cor:
       root.left = insertRec(root.left,data,level + 1)
    else:
       root.right = insertRec(root.right,data,level + 1)
    return root
#==============================================================================

#==============================================================================
def searchRec(root,point,level):
    if root is None:
       return None
    check = level % 3
    root_cor = 0.0
    data_cor = 0.0

    if checkPoint(root.data,point) is True:
       return root

    #gen_data of root_cor and data_cor    
    if check == 0:
       root_cor = root.data.x
       data_cor = point.x
    elif check == 1:
       root_cor = root.data.y
       data_cor = point.y
    else:
       root_cor = root.data.z
       data_cor = point.z
    
    if root_cor > data_cor:
       return searchRec(root.left,point,level + 1)
    return searchRec(root.right,point,level + 1)
#==============================================================================

#==============================================================================
def minNode(root,left,right,level):
    res = root
    check = level % 3
    if check == 0:
       if left != None and left.data.x < res.data.x:
          res = left
       if right != None and right.data.x < res.data.x:
          res = right
    elif check == 1:
       if left != None and left.data.y < res.data.y:
          res = left
       if right != None and right.data.y < res.data.y:
          res = right
    else:
       if left != None and left.data.z < res.data.z:
          res = left
       if right != None and right.data.z < res.data.z:
          res = right
    return res
#==============================================================================

#==============================================================================
def maxNode(root,left,right,level):
    res = root
    check = level % 3
    if check == 0:
       if left != None and left.data.x > res.data.x:
          res = left
       if right != None and right.data.x > res.data.x:
          res = right
    elif check == 1:
       if left != None and left.data.y > res.data.y:
          res = left
       if right != None and right.data.y > res.data.y:
          res = right
    else:
       if left != None and left.data.z > res.data.z:
          res = left
       if right != None and right.data.z > res.data.z:
          res = right
    return res
#==============================================================================

#==============================================================================
#min point of level d:
def findMinRec(root,d,level):
    if root is None:
       return None
    check = level % 3
    if check == d:
       if root.left == None:
          return root
       return findMinRec(root.left,d,level + 1)    
    return minNode(root,findMinRec(root.left,d,level + 1),findMinRec(root.right,d,level + 1),d)
#==============================================================================

#==============================================================================
#min point of level d:
def findMaxRec(root,d,level):
    if root is None:
       return None
    check = level % 3
    if check == d:
       if root.right == None:
          return root
       return findMaxRec(root.right,d,level + 1)    
    return maxNode(root,findMaxRec(root.left,d,level + 1),findMaxRec(root.right,d,level + 1),d)
#==============================================================================

#==============================================================================
#bugs
def deleteRec(root,point,level):
    if root == None:
       return None
    check = level % 3

    #if current_node is point
    if checkPoint(root.data,point) is True:
       if root.right != None:
          min = findMinRec(root.right,check,level + 1)

          #copy point
          root.data.x = min.data.x
          root.data.y = min.data.y
          root.data.z = min.data.z

          root.right = deleteRec(root.right,min.data,level + 1)
       elif root.left != None:
          min = findMaxRec(root.left,check,level + 1)

          #copy point
          root.data.x = min.data.x
          root.data.y = min.data.y
          root.data.z = min.data.z

          root.right = deleteRec(root.left,min.data,level + 1)
          root.left = None
       else: #leaf node
          root = None
       return root

    #not contain point
    root_cor = 0.0
    data_cor = 0.0

    if check == 0:
       root_cor = root.data.x
       data_cor = point.x
    elif check == 1:
       root_cor = root.data.y
       data_cor = point.y
    else:
       root_cor = root.data.z
       data_cor = point.z
    
    if root_cor > data_cor:
       root.left = deleteRec(root.left,point,level + 1)
    else:
       root.right = deleteRec(root.right,point,level + 1)
    return root

#==============================================================================

#==============================================================================
def print2DUtil(root,space = 0) -> None:
    if root is None:
       return None
    space += 10
    if root.right is not None:
       print2DUtil(root.right,space)
    
    print("\n{}{},{},{}\n".format(' '*(space - 10),root.data.x,root.data.y,root.data.z))

    if root.left is not None:
       print2DUtil(root.left,space)
#==============================================================================

#==============================================================================
def clearRec(root) -> None:
    if root is None:
       return
    clearRec(root.left)
    clearRec(root.right)
    del root
#==============================================================================

#==============================================================================
def findNNRec(root,query_point,best_node,best_distance,depth) -> None:
    if not root:
       return best_distance
    dist = euclide_distance(root.data,query_point)

    if dist < best_distance:
       best_node.append(root)
       best_distance = dist
  
    check = depth % 3
    root_cor = 0.0
    data_cor = 0.0

    if check == 0:
       root_cor = root.data.x
       data_cor = query_point.x
    elif check == 1:
       root_cor = root.data.y
       data_cor = query_point.y
    else:
       root_cor = root.data.z
       data_cor = query_point.z

    if data_cor < root_cor:
       good_side = root.left
       bad_side = root.right
    else:
       good_side = root.right
       bad_side = root.left

    best_distance = findNNRec(good_side,query_point,best_node,best_distance,depth + 1)
    if abs(root_cor - data_cor) < best_distance:
       best_distance = findNNRec(bad_side,query_point,best_node,best_distance,depth + 1)
    return best_distance
    
#==============================================================================

#KDtree definition

In [ ]:
class KdTree:
      def __init__(self):
          self.root = None
          self.size = 0
      
      def __del__(self):
          self.clear()

      def clear(self):
          clearRec(self.root)
          self.root = None
          self.size = 0
      
      def insert(self,data):
          self.root = insertRec(self.root,data,0)
          self.size += 1
      
      def search(self,point):
          node = searchRec(self.root,point,0)
          if node is None:
             return -1
          return node.data

      def findMin(self,d):
          return findMinRec(self.root,d,0)

      def delete(self,point):
          if self.search(point) == None:
             print("no element in Kd tree")
          else:
             self.root = deleteRec(self.root,point,0)
             self.size -= 1

      def buildKdTree(self,points):
          for point in points:
              self.insert(point)
      
      def buildKdTree_pre(self,points,begin,end):
          if begin >= end:
             return None
          mid = int((begin + end)/2)
          self.insert(points[mid])
          self.buildKdTree_pre(points,begin,mid)
          self.buildKdTree_pre(points,mid + 1,end) 

      def findkNN(self,query_point,k):
          stack = []
          findNNRec(self.root,query_point,stack,np.infty,0)
          for i in range(k):
              if len(stack) >= 1:
                 point = stack.pop().data
                 print("point{}: {},{},{} with euclide_distance: {}\n".format(i + 1,point.x,point.y,point.z,euclide_distance(query_point,point)))

      def getSize(self):
          return self.size
      
      def getRoot(self):
          return self.root
      
      def printTree(self):
          print2DUtil(self.root)

#Main

In [ ]:
def build_method(points,method = "normal"):
     if method == "re_balance":
        preprocess(points,0,0,len(points))
        model = KdTree()
        model.buildKdTree_pre(points,0,len(points))
     else:
        model = KdTree()
        model.buildKdTree(points)
     return model

model = build_method(points,"re_balance")

In [ ]:
for i in range(10):
    point_temp = model.search(points[i])
    if point_temp != -1:
      print("{},{},{}\n".format(point_temp.x,point_temp.y,point_temp.z))

23.131,52.127,7.559

11.961,57.057,58.753

37.077,69.891,19.278

40.769,97.249,90.329

6.113,75.402,95.218

40.777,94.253,64.774

45.31,23.791,34.663

98.692,8.89,98.176

85.745,39.208,14.013

90.202,93.612,26.914



In [ ]:
model.findkNN(Point(1,2,3),3)

point1: 23.131,52.127,7.559 with euclide_distance: 54.98437751761859

point2: 37.077,69.891,19.278 with euclide_distance: 78.58569268002924

point3: 40.777,94.253,64.774 with euclide_distance: 117.93579954365002



In [ ]:
model.clear()
model.getRoot()

In [ ]:
model.printTree()